# 1 Paradigmas

Basado en A Discrete-Event Simulation Course Based on the SimPy Language [Ref](http://heather.cs.ucdavis.edu/~matloff/SimCourse/PLN/DESimIntro.pdf)

## 1.1 Paradigma orientado a la actividad


## Sistema de colas

* Los trabajos llegan en momentos aleatorios y el servidor de trabajos toma un tiempo aleatorio para cada servicio. 
* El tiempo transcurrido entre la llegada de los trabajos y el tiempo necesario para servir un trabajo será
variables aleatorias continuas, posiblemente con distribuciones exponenciales u otras distribuciones continuas.

Ejemplo: el servidor es un cajero automático y los trabajos son clientes que esperan en la cola.

Bajo el paradigma orientado a la actividad, dividiríamos el tiempo en pequeños incrementos. Si por ejemplo la media
del tiempo entre llegadas fue, 20 segundos, podríamos dividir el tiempo en incrementos de tamaño 0.001. En cada
punto, nuestro código miraría todas las actividades, p. ej. servicio de trabajo actualmente activo, y verificaría la
ocurrencia de eventos, p.ej. finalización del servicio.

Nuestro objetivo es encontrar la espera promedio de trabajo.

<img src = "../imgs/act_orien_sim.png">


## 1.2 Paradigma orientado a la entidad

Un programa de simulación orientado a la actividad será muy lento de ejecutar. La mayoría de los incrementos de tiempo no producirá ningún cambio de estado en el sistema, es decir, no habrá nuevas llegadas a la cola y no se completarán servicio por parte del servidor. Por lo tanto, las comprobaciones de actividad serán tiempo de procesador perdido

Sin embargo, la inspección del pseudocódigo anterior muestra una forma de aumentar drásticamente la velocidad de simulación. En lugar de tener el tiempo "arrastrándose" tan lentamente, ¿por qué no tomar un "atajo" para el próximo evento?

Por ejemplo, si reemplazamos 

1 for SimTime = 1*0.001 to NIncrements*0.001 do

Por el momento del próximo evento:

<img src = "../imgs/entity_orient_parad_segment.png">

Con esta modificación, revisamos nuevamente el pseudocogido

<img  src="../imgs/entity_orient_parad.png">

Una parte importante del tiempo de ejecución del programa consistirá en una operación de búsqueda mínima dentro del conjunto de eventos.

## 1.3 Paradigma orientado a procesos

Aquí cada actividad de simulación es modelada por un proceso. Para el caso anterior, tendríamos dos hilos, uno que simula las llegadas y otro, para el funcionamiento del servidor. 

Hilo de llegadas

<img src="../imgs/thread1.png">

Hilo del servidor

<img src="../imgs/thread2.png">

Hilo administrador del conjunto de eventos

<img src="../imgs/event_set_thread.png">

Función main()

<img src="../imgs/main.png">

# Simpy overview

### Clases

**Proceso:** simula una entidad que evoluciona en el tiempo, p. un cliente que necesita ser atendido por un cajero automático (nos referiremos a él como un hilo, aunque no sea un hilo formal de Python)

**Recurso:** simula algo para poner en cola, p. Ej. la máquina

### Funciones

**activate():** se utiliza para marcar un hilo como ejecutable cuando se crea por primera vez

**simulate():** inicia la simulación

**yield hold:** se utiliza para indicar el paso de una cierta cantidad de tiempo dentro de un hilo; **yield** es un operador de Python cuyo primer operando es una función a llamar, en este caso un código para una función que realiza
la operación de retención en la biblioteca SimPy

**yield request:** se usa para hacer que un hilo se una a una cola para un recurso determinado (y comience a usarlo inmediatamente si no hay otros trabajos esperando el recurso)

**yield release:**  se utiliza para indicar que el subproceso se realiza utilizando el recurso dado, lo que permite que el siguiente subproceso en la cola, si lo hay, use el recurso

**yield passivate:** hace que un hilo espere hasta que "otro hilo" lo "despierte".

**reactivate:** hace "despertar" a un hilo previamente pasivo

**cancel():** cancela todos los eventos asociados con un hilo previamente pasivo

In [8]:
!pip install SimPy

In [11]:
import SimPy

ModuleNotFoundError: No module named 'SimPy'

In [9]:
#!/usr/bin/env python

# MachRep1.py

# Introductory SimPy example: Two machines, which sometimes break down.
# Up time is exponentially distributed with mean 1.0, and repair time is
# exponentially distributed with mean 0.5. There are two repairpersons,
# so the two machines can be repaired simultaneously if they are down
# at the same time.

# Output is long-run proportion of up time. Should get value of about
# 0.66.

import SimPy.Simulation # required
import random

class G: # global variables
    Rnd = random.Random(12345)

class MachineClass(SimPy.Simulation.Process):
    UpRate = 1/1.0 # reciprocal of mean up time
    RepairRate = 1/0.5 # reciprocal of mean repair time
    TotalUpTime = 0.0 # total up time for all machines
    NextID = 0 # next available ID number for MachineClass objects
    
    def __init__(self): # required constructor
        SimPy.Simulation.Process.__init__(self) # must call parent constructor
        # instance variables
        self.StartUpTime = 0.0 # time the current up period started
        self.ID = MachineClass.NextID # ID for this MachineClass object
        MachineClass.NextID += 1
    def Run(self): # required constructor
        while 1:
            # record current time, now(), so can see how long machine is up
            self.StartUpTime = SimPy.Simulation.now()
            # hold for exponentially distributed up time
            UpTime = G.Rnd.expovariate(MachineClass.UpRate)
            yield SimPy.Simulation.hold,self,UpTime # simulate UpTime
            # update up time total
            MachineClass.TotalUpTime += SimPy.Simulation.now() - self.StartUpTime
            RepairTime = G.Rnd.expovariate(MachineClass.RepairRate)
            # hold for exponentially distributed repair time
            yield SimPy.Simulation.hold,self,RepairTime

def main():
    SimPy.Simulation.initialize() # required
    # set up the two machine threads
    for I in range(2):
        # create a MachineClass object
        M = MachineClass()
        # register thread M, executing M’s Run() method,
        SimPy.Simulation.activate(M,M.Run()) # required
        # run until simulated time 10000
    MaxSimtime = 10000.0
    SimPy.Simulation.simulate(until=MaxSimtime) # required
    print ("the percentage of up time was", MachineClass.TotalUpTime/(2*MaxSimtime))

if __name__ == '__main__': main()

ModuleNotFoundError: No module named 'SimPy'